In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingRandomSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
import os
import graphviz
from sklearn import tree
from sklearn.model_selection import GridSearchCV


In [2]:
# diretorio
diretorio = r'C:\Users\Computadores Gamer\OneDrive\Área de Trabalho\mba esalq\arvores_redes_ensemble_models'
diretorio = diretorio.replace('\\','/')
os.chdir(diretorio)
os.listdir()

['aula1',
 'aula2',
 'aula3',
 'aula4',
 'aulas_principal.doc',
 'exercicios_arvore_decisao.py',
 'exercicio_arvore_regressao.py',
 'exercicio_random_forest.py',
 'script_arvore.py',
 'script_har.py',
 'titanic.xlsx',
 'voos.py']

# Significado das váriaveis
passengerid = ID do passageiro do navio (código primário);

survived = Se sobreviveu ao naufrágio estará como 1 e caso esteja com 0 (zero) não sobreviveu;

pclass = Tipo de classe de passagem (Do 1 ao 3), sendo 1 a melhor classe e 3 a pior classe;

name = Nome do passageiro;

sex = Gênero do passageiro, sendo masculino e feminino;

age = Idade do passageiro na data da ocorrência do naufrágio;

sibsp = Número de irmãos / cônjuges a bordo;

parch = Número de pais / filhos a bordo;

ticket = Código do ticket;

fare = Valor da passagem;

cabin = Código de identificação da Cabine;

embarked = Local ondem o passageiro embarcou no navio;

In [3]:
np.random.seed(123)

In [20]:
# dados
dados = pd.read_excel('titanic.xlsx', sheet_name='Sheet 1')
dados = pd.DataFrame(dados)
dados.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    object 
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB


array(['S', 'C', 'Q'], dtype=object)

In [17]:
dummies_survived = pd.get_dummies(dados['Survived'], prefix='survived')
dummies_sex = pd.get_dummies(dados['Sex'], prefix='Sex')
dummies_embarked = pd.get_dummies(dados['Embarked'], prefix='Embarked')

dados = pd.concat([dados, dummies_survived,dummies_sex,dummies_embarked], axis=1)
dados

# sugestao do plantao: transformar essas variaveis acima em dummies e uma coluna pra cada categoria de cada variavel 
# lembrar de apagar a coluna original: ex: vai ficar survivedN e outra coluna survivedS
# na formula da arvore nao precisa por as duas dummies, pq a informacao é a mesma







,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,survived_N,survived_Y,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,N,3,male,22.000000,1,0,7.2500,S,1,0,0,1,0,0,1
1,Y,1,female,38.000000,1,0,71.2833,C,0,1,1,0,1,0,0
2,Y,3,female,26.000000,0,0,7.9250,S,0,1,1,0,0,0,1
3,Y,1,female,35.000000,1,0,53.1000,S,0,1,1,0,0,0,1
4,N,3,male,35.000000,0,0,8.0500,S,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,N,2,male,27.000000,0,0,13.0000,S,1,0,0,1,0,0,1
887,Y,1,female,19.000000,0,0,30.0000,S,0,1,1,0,0,0,1
888,N,3,female,29.699118,1,2,23.4500,S,1,0,1,0,0,0,1
889,Y,1,male,26.000000,0,0,30.0000,C,0,1,0,1,1,0,0


In [ ]:
# separação base treino e teste
var_x = dados[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
var_y = dados[['Survived']]

treino_x, teste_x, treino_y, teste_y = train_test_split(var_x, var_y, test_size=0.2, random_state=123)


In [ ]:
# criacao da arvore
cp = 0.01
arvore = DecisionTreeClassifier (criterion='gini',
                                 ccp_alpha=cp,
                                 max_depth = 5)

In [ ]:
# fit 
arvore.fit(treino_x, treino_y)


In [ ]:
tree.plot_tree(arvore)

In [ ]:
# avaliacao base teste
predicts = arvore.predict(teste_x)
matriz_confusao = confusion_matrix(teste_y, predicts)
acuracia_teste = accuracy_score(teste_y, predicts)
acuracia_teste

In [ ]:
# tunning ccp
limite_inf = 0.01
limite_sup = 0.001
hiperparametros = {'ccp_alpha':(limite_inf,limite_sup),
                   'max_depth':(1,10)}

grid = HalvingRandomSearchCV(estimator = arvore,
                             param_distributions= hiperparametros,
                             scoring='accuracy',
                             factor=2,
                             n_jobs=1,
                             cv=5)
grid.fit(treino_x, treino_y)


In [ ]:
# melhor ccp_alpha
ccp_otimo = grid.best_params_['ccp_alpha']


In [ ]:
# melhor max_depth
max_otimo = grid.best_params_['max_depth']
max_otimo

In [ ]:
arvore2 = DecisionTreeClassifier (criterion='gini',
                                 ccp_alpha = ccp_otimo,
                                 max_depth = max_otimo)

In [ ]:
# fit arvore tunada
arvore2.fit(treino_x, treino_y)

In [ ]:
tree.plot_tree(arvore2)

In [ ]:
# avaliacao base teste de arvore podada
predicts2 = arvore2.predict(teste_x)
matriz2 = confusion_matrix(teste_y, predicts2)
acuracia2 = accuracy_score(teste_y, predicts2) 

In [ ]:
matriz2

In [ ]:
acuracia2